## Sustainable and Entrepreneurial Finance- Homework #1

In [1]:
#Necessary libraries to run the code
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


### Question 3- Optimizing the portfolio

In [3]:
from scipy.optimize import minimize

file_path = r"data_h1\filtered_data\DS_RI_USD_M.xlsx"

#Filtering the data frame and using data only from 2014 to 2024
df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)
df_returns = df_returns.select_dtypes(include=[np.number])
df_returns.columns = pd.to_datetime(df_returns.columns, format='%Y-%m-%d', errors='coerce')
df_returns=df_returns.loc[:, "2014-01-31":"2024-12-31"]

#Computing the mean and the covariance matrix 
mu_hat = df_returns.mean(axis=1,skipna=True)
Sigma_hat = df_returns.T.cov()

# Number of assets
n_assets = len(mu_hat)

# Convert expected returns and covariance matrix to NumPy arrays
mu_array = mu_hat.values.reshape(-1, 1)  # Column vector
Sigma_array = Sigma_hat.values  # Covariance matrix

# Constraint: sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bounds: No short selling (weights >= 0)
bounds = [(0, 1) for _ in range(n_assets)]

### Step 1: Compute Minimum Variance Portfolio (MVP)
def portfolio_variance(w, Sigma):
    return w.T @ Sigma @ w

# Initial guess (equal weights)
w0 = np.ones(n_assets) / n_assets

# Solve for minimum variance portfolio
Sigma_array = np.nan_to_num(Sigma_array)  #remove Nan for non-overlapping returns for some companies

mvp_result = minimize(portfolio_variance, w0, args=(Sigma_array,), method='SLSQP', bounds=bounds, constraints=constraints)
w_mvp = mvp_result.x  # Optimal weights

# Compute return and variance of the MVP
mu_mvp = w_mvp.T @ mu_array
sigma_mvp = np.sqrt(mvp_result.fun)

# Compute Maximum Return Portfolio
def negative_return(w, mu):
    return -w.T @ mu

max_return_result = minimize(negative_return, w0, args=(mu_array,), method='SLSQP', bounds=bounds, constraints=constraints)
w_max_ret = max_return_result.x  # Optimal weights

# Compute return and variance of the max return portfolio
mu_max_ret = w_max_ret.T @ mu_array
sigma_max_ret = np.sqrt(w_max_ret.T @ Sigma_array @ w_max_ret)

## Compute Efficient Frontier (10 to 20 portfolios)
# Define target returns between mu_mvp and mu_max_ret (increments of 0.5%)
target_returns = np.linspace(mu_mvp, mu_max_ret, num=15)  

efficient_frontier = []

for target_return in target_returns:
    # Constraint: Portfolio return = target return
    return_constraint = {'type': 'eq', 'fun': lambda w: w.T @ mu_array - target_return}
    
    # Solve optimization problem
    result = minimize(portfolio_variance, w0, args=(Sigma_array,), method='SLSQP', bounds=bounds, constraints=[constraints, return_constraint])
    
    # Store results
    w_optimal = result.x
    sigma_optimal = np.sqrt(result.fun)
    efficient_frontier.append((target_return, sigma_optimal, w_optimal))

# Convert results to a DataFrame for visualization
efficient_frontier_df = pd.DataFrame(efficient_frontier, columns=['Target Return', 'Risk (Std Dev)', 'Weights'])

# Display results
display(efficient_frontier_df)

# Save the efficient frontier DataFrame to an Excel file
efficient_frontier_df.to_excel("data_h1/results/efficient_frontier.xlsx", index=False)

C:\Users\ameli\AppData\Local\Temp\ipykernel_38716\1174091648.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_returns = pd.read_excel(file_path, sheet_name="MR raw", index_col=0, parse_dates=True)


,Target Return,Risk (Std Dev),Weights
0,[0.00282732133311229],0.024176,"[0.0, 8.792963562271686e-19, 3.999549939155198..."
1,[0.004911081652585839],0.024254,"[8.30100216350049e-18, 0.0, 0.0, 4.53263129268..."
2,[0.006994841972059387],0.025557,"[5.32141483644484e-18, 1.630586840232727e-18, ..."
3,[0.009078602291532935],0.026727,"[5.961187601446956e-18, 0.0, 4.8849467396779e-..."
4,[0.011162362611006484],0.028449,"[3.8279309840179336e-18, 0.0, 0.0, 5.343233097..."
5,[0.013246122930480032],0.030717,"[2.010097747018624e-18, 9.492259867751116e-19,..."
6,[0.01532988324995358],0.033567,"[2.701319952207758e-18, 1.2791842128261991e-18..."
7,[0.01741364356942713],0.037557,"[4.200249344514363e-18, 0.0, 0.0, 1.2134465374..."
8,[0.01949740388890068],0.043090,"[0.0, 4.487821380716247e-18, 9.878065795295582..."
9,[0.021581164208374227],0.050269,"[1.3348133271384778e-17, 0.0, 5.85272728069945..."


In [6]:
# Extract the monthly returns data
monthly_returns = df_returns.copy()

monthly_returns= np.nan_to_num(monthly_returns)#removing Nan and replacing them with 0 to get the multiplications correctly 

# Initialize a dictionary to store the ex-post returns for each portfolio
ex_post_returns = {}
idx=1

# Iterate over each portfolio in the efficient frontier
for weights in efficient_frontier_df["Weights"]:
    # Compute the portfolio return for each month
    weights = np.array(weights).reshape(1, -1)
    # Store the time series of ex-post returns
    portfolio_returns = np.dot(weights,monthly_returns)
    ex_post_returns[f'Portfolio_{idx}'] = portfolio_returns.flatten()
    idx=idx+1

# Convert the dictionary to a DataFrame for easier analysis
ex_post_returns_df = pd.DataFrame(ex_post_returns, index=df_returns.T.index)

# Display the ex-post returns DataFrame
display(ex_post_returns_df)
ex_post_returns_df.to_excel("data_h1/results/ex_post_returns.xlsx")

,Portfolio_1,Portfolio_2,Portfolio_3,Portfolio_4,Portfolio_5,Portfolio_6,Portfolio_7,Portfolio_8,Portfolio_9,Portfolio_10,Portfolio_11,Portfolio_12,Portfolio_13,Portfolio_14,Portfolio_15
2014-01-31,-0.014696,-0.012530,-0.016660,-0.017014,-0.017357,-0.018263,-0.020471,-0.030503,-0.050905,-0.050975,-0.059191,-0.082286,-0.120253,-0.172814,-0.253461
2014-02-28,0.051545,0.051319,0.046274,0.044808,0.041150,0.037201,0.037509,0.036746,0.036149,0.036947,0.034925,0.029449,0.024936,0.021664,-0.031812
2014-03-31,0.006410,0.011792,0.017367,0.017572,0.016595,0.013997,0.007563,-0.002388,-0.005237,-0.011608,-0.014504,-0.009633,0.000475,0.018721,0.045731
2014-04-30,0.012851,0.014110,0.019507,0.022093,0.021800,0.017635,0.016739,0.021155,0.026706,0.031664,0.037812,0.047083,0.061138,0.072632,0.118324
2014-05-30,0.016030,0.018625,0.018505,0.024227,0.030497,0.031523,0.033470,0.035314,0.037426,0.033631,0.029858,0.029369,0.033161,0.032007,0.023731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.023300,0.024797,0.032751,0.036302,0.039559,0.046536,0.048359,0.046882,0.044405,0.039351,0.038363,0.045581,0.055498,0.060701,0.049486
2024-09-30,0.001720,-0.001686,0.000801,0.000159,0.000043,-0.003563,-0.008503,-0.009890,-0.006375,-0.004585,-0.006098,-0.007968,-0.006924,0.008179,0.048916
2024-10-31,0.004102,0.007840,0.003122,0.001577,0.001313,0.001806,0.001301,0.003576,0.003027,-0.000310,0.000050,0.007541,0.017320,0.037387,0.074073
2024-11-29,-0.006616,-0.000765,0.005198,0.008929,0.013927,0.016213,0.024982,0.043489,0.060115,0.080071,0.107161,0.139322,0.172330,0.192775,0.168022
